In [1]:
import argparse

import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
from dgl.dataloading import (
    as_edge_prediction_sampler,
    DataLoader,
    MultiLayerFullNeighborSampler,
    negative_sampler,
    NeighborSampler,
)
from ogb.linkproppred import DglLinkPropPredDataset, Evaluator

In [3]:
def to_bidirected_with_reverse_mapping(g):
    g_simple, mapping = dgl.to_simple(
        dgl.add_reverse_edges(g), return_counts="count", writeback_mapping=True
    )
    c = g_simple.edata["count"]
    num_edges = g.num_edges()
    mapping_offset = torch.zeros(
        g_simple.num_edges() + 1, dtype=g_simple.idtype
    )
    mapping_offset[1:] = c.cumsum(0)
    idx = mapping.argsort()
    idx_uniq = idx[mapping_offset[:-1]]
    reverse_idx = torch.where(
        idx_uniq >= num_edges, idx_uniq - num_edges, idx_uniq + num_edges
    )
    reverse_mapping = mapping[reverse_idx]
    src1, dst1 = g_simple.edges()
    src2, dst2 = g_simple.find_edges(reverse_mapping)
    assert torch.equal(src1, dst2)
    assert torch.equal(src2, dst1)
    return g_simple, reverse_mapping

In [6]:
#dataset = DglLinkPropPredDataset("ogbl-citation2",root="/raid/bear/raw_dataset")
#dataset = DglLinkPropPredDataset("ogbl-ppa",root="/raid/bear/raw_dataset")
DglLinkPropPredDataset("ogbl-vessel",root="/raid/bear/raw_dataset")
g = dataset[0]
g, reverse_eids = to_bidirected_with_reverse_mapping(g)
edge_split = dataset.get_edge_split()

Downloaded 0.11 GB: 100%|██████████| 118/118 [08:35<00:00,  4.37s/it]


Extracting /raid/bear/raw_dataset/vessel.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 7049.25it/s]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00, 11.36it/s]

Saving...


In [5]:
g

Graph(num_nodes=576289, num_edges=42463862,
      ndata_schemes={'feat': Scheme(shape=(58,), dtype=torch.int64)}
      edata_schemes={'count': Scheme(shape=(), dtype=torch.int64)})